In [1]:
import sys
from fastai.text.all import * 
from fastai import *

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-0ff0723693d7>", line 2, in <module>
    from fastai.text.all import *
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/fastai/text/all.py", line 1, in <module>
    from ..basics import *
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/fastai/basics.py", line 1, in <module>
    from .data.all import *
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/fastai/data/all.py", line 1, in <module>
    from ..torch_basics import *
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/fastai/torch_basics.py", line 1, in <module>
    from torch import multiprocessing
  File "/anaconda/envs/fastai2/lib/python3.8/site-packages/torch/__init__.py", line 189, in <module>
    from torch._C import *
ImportError: numpy.core.multiarray failed t

TypeError: object of type 'NoneType' has no len()

In [3]:
from fastai.text.all import *
from pathlib import Path
import pandas as pd
from random import choice
import re
import json

def get_next_word(learn, text, no_unk=True, temperature=1., decoder=decode_spec_tokens):
    learn.model.reset()
    idxs = idxs_all = learn.dls.test_dl([text]).items[0]
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    preds,_ = learn.get_preds(dl=[(idxs[None],)])
    res = preds[0][-1]
    if no_unk: res[unk_idx] = 0.
    if temperature != 1.: res.pow_(1 / temperature)
    idx = torch.multinomial(res, 3)
    num = learn.dls.train_ds.numericalize
    tokens = [num.vocab[i] for i in idx if num.vocab[i] not in [BOS, PAD]]
    return decoder(tokens)

def beam_search_modified(learn, text:str, confidence:float, no_unk:bool=True, no_punct:bool=True, top_k:int=10, beam_sz:int=100, temperature:float=1.,
                    sep:str=' ', decoder=decode_spec_tokens):
    learn.model.reset()
    learn.model.eval()
    idx = learn.dls.test_dl([text]).items[0][None]
    nodes = None
    nodes = idx.clone()
    scores = idx.new_zeros(1).float()
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    

    with torch.no_grad():
        while(torch.exp(-scores[0])>confidence):
            out = F.log_softmax(learn.model(idx)[0][:,-1], dim=-1)
            
            if no_unk: out[:, unk_idx] = -float('Inf')

            values, indices = out.topk(top_k, dim=-1)
            
            scores = (-values + scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]

            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                                    indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]            
            learn.hidden = [(h[0][:,indices_idx[sort_idx],:],h[1][:,indices_idx[sort_idx],:]) for h in learn.model[0].hidden]
            idx = nodes[:,-1][:,None]



        if temperature != 1.: scores.div_(temperature)
        node_idx = torch.multinomial(torch.exp(-scores), 1).item()
        num = learn.dls.train_ds.numericalize
        tokens = [num.vocab[i] for i in nodes[node_idx][1:] if num.vocab[i] not in [BOS, PAD]]
        sep = learn.dls.train_ds.tokenizer.sep
        return sep.join(decoder(tokens))

def beam_search(learn, text:str, n_words:int, no_unk:bool=True, top_k:int=10, beam_sz:int=100, temperature:float=1.,
                    sep:str=' ', decoder=decode_spec_tokens):
    learn.model.reset()
    learn.model.eval()
    idx = learn.dls.test_dl([text]).items[0][None]
    nodes = None
    nodes = idx.clone()
    scores = idx.new_zeros(1).float()
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    with torch.no_grad():
        for k in progress_bar(range(n_words), leave=False):
            out = F.log_softmax(learn.model(idx)[0][:,-1], dim=-1)
            if no_unk: out[:, unk_idx] = -float('Inf')
            values, indices = out.topk(top_k, dim=-1)
            scores = (-values + scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]
            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                                    indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]
            learn.hidden = [(h[0][:,indices_idx[sort_idx],:],h[1][:,indices_idx[sort_idx],:]) for h in learn.model[0].hidden]
            idx = nodes[:,-1][:,None]
        if temperature != 1.: scores.div_(temperature)
        node_idx = torch.multinomial(torch.exp(-scores), 1).item()
        num = learn.dls.train_ds.numericalize
        tokens = [num.vocab[i] for i in nodes[node_idx][1:] if num.vocab[i] not in [BOS, PAD]]
        sep = learn.dls.train_ds.tokenizer.sep
        return sep.join(decoder(tokens))


In [13]:
path = URLs.AMAZON_REVIEWS

In [4]:
df_train = pd.read_csv('/home/advaitmb/.fastai/data/amazon_review_full_csv/train.csv', header=None, names=['rating', 'review_title', 'review']); df_train.head()

,rating,review_title,review
0,3,more like funchuck,"Gave this to my dad for a gag gift after directing ""Nunsense,"" he got a reall kick out of it!"
1,5,Inspiring,"I hope a lot of people hear this cd. We need more strong and positive vibes like this. Great vocals, fresh tunes, cross-cultural happiness. Her blues is from the gut. The pop sounds are catchy and mature."
2,5,The best soundtrack ever to anything.,"I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."
3,4,Chrono Cross OST,"The music of Yasunori Misuda is without question my close second below the great Nobuo Uematsu.Chrono Cross OST is a wonderful creation filled with rich orchestra and synthesized sounds. While ambiance is one of the music's major factors, yet at times it's very uplifting and vigorous. Some of my favourite tracks include; ""Scars Left by Time, The Girl who Stole the Stars, and Another World""."
4,5,Too good to be true,Probably the greatest soundtrack in history! Usually it's better to have played the game first but this is so enjoyable anyway! I worked so hard getting this soundtrack and after spending [money] to get it it was really worth every penny!! Get this OST! it's amazing! The first few tracks will have you dancing around with delight (especially Scars Left by Time)!! BUY IT NOW!!


In [34]:
(set(df_train.rating))

{1, 2, 3, 4, 5}

In [7]:
TextDataLoaders.from_df??

In [10]:
dls = TextDataLoaders.from_df(
    df_train,
    path='path',
    valid_pct=0.2,
    seed=None,
    text_col='review',
    label_col='rating',
    label_delim=None,
    y_block=None,
    text_vocab=None,
    is_lm=True,
    valid_col=None,
    tok_tfm=None,
    seq_len=72,
    backwards=False,
    bs=32,
    val_bs=None,
    shuffle_train=True,
    device=None,
)

/anaconda/envs/fastai2/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
dls.show_batch()

,text,text_
0,"xxbos i have no xxunk with the book itself . xxmaj xxunk a copy for a long time . xxmaj however , i purchased a used book with the xxunk that it was a "" xxunk xxmaj copy . "" xxmaj the copy i received xxup was xxup not xxunk . xxmaj this is a xxunk . xxmaj if you xxunk it as an xxunk copy , do n't xxunk one that","i have no xxunk with the book itself . xxmaj xxunk a copy for a long time . xxmaj however , i purchased a used book with the xxunk that it was a "" xxunk xxmaj copy . "" xxmaj the copy i received xxup was xxup not xxunk . xxmaj this is a xxunk . xxmaj if you xxunk it as an xxunk copy , do n't xxunk one that is"
1,i 've had to xxunk xxunk on the how - to xxunk . xxmaj and xxunk the xxup vcr xxunk is xxunk -- you ca n't xxunk the start and end time but have to xxunk ( xxunk xxmaj i 'm xxunk something ) … but those are my only xxunk . xxmaj this is a good product for your xxrep 3 $ . xxbos i ordered the cake topper xxmaj xxunk,'ve had to xxunk xxunk on the how - to xxunk . xxmaj and xxunk the xxup vcr xxunk is xxunk -- you ca n't xxunk the start and end time but have to xxunk ( xxunk xxmaj i 'm xxunk something ) … but those are my only xxunk . xxmaj this is a good product for your xxrep 3 $ . xxbos i ordered the cake topper xxmaj xxunk xxunk
2,"their country 's xxunk of xxunk for their xxunk while also xxunk their own xxunk as xxmaj xxunk 's "" xxunk xxup xxunk "" . xxmaj like our own young people , they are xxunk with some of the xxunk of "" xxunk culture "" - xxmaj xxunk xxunk , rock xxunk , a culture of xxunk , etc . xxmaj what they seem to be xxunk , however , is a","country 's xxunk of xxunk for their xxunk while also xxunk their own xxunk as xxmaj xxunk 's "" xxunk xxup xxunk "" . xxmaj like our own young people , they are xxunk with some of the xxunk of "" xxunk culture "" - xxmaj xxunk xxunk , rock xxunk , a culture of xxunk , etc . xxmaj what they seem to be xxunk , however , is a xxunk"
3,"very annoying after xxunk . xxmaj the xxunk are very nice and xxunk and it does xxunk your xxunk to use her xxunk and to be creative , but all in all , i think the game was xxunk too xxunk for what it is . xxmaj after having this in our house for 4 xxunk , my xxunk have xxunk rather xxunk with it . xxbos xxmaj tess xxmaj lindsay has","annoying after xxunk . xxmaj the xxunk are very nice and xxunk and it does xxunk your xxunk to use her xxunk and to be creative , but all in all , i think the game was xxunk too xxunk for what it is . xxmaj after having this in our house for 4 xxunk , my xxunk have xxunk rather xxunk with it . xxbos xxmaj tess xxmaj lindsay has xxunk"
4,the xxunk see 's xxmaj tess ? xxmaj read the book to find out . xxbos xxmaj this xxmaj rapunzel game was not really my xxunk game . xxmaj it was n't that much fun . xxmaj you have to wait a long time to xxunk to another xxunk . xxmaj and xxunk you want to go xxunk xxunk you have to xxunk xxunk up and down . xxmaj this game was,xxunk see 's xxmaj tess ? xxmaj read the book to find out . xxbos xxmaj this xxmaj rapunzel game was not really my xxunk game . xxmaj it was n't that much fun . xxmaj you have to wait a long time to xxunk to another xxunk . xxmaj and xxunk you want to go xxunk xxunk you have to xxunk xxunk up and down . xxmaj this game was not
5,"it seems xxunk and is xxunk xxunk to read cds , xxunk the time xxunk . i am going to try to return it to xxmaj amazon . xxbos xxmaj while i am still in the xxunk of learning the xxmaj yoruba xxunk , i think that this book is a nice xxunk tool . xxmaj please keep in xxunk that the new book comes with to xxunk cd 's that are","seems xxunk and is xxunk xxunk to read cds , xxunk the time xxunk . i am going to try to return it to xxmaj amazon . xxbos xxmaj while i am still in the xxunk of learning the xxmaj yoruba xxunk , i think that this book is a nice xxunk tool . xxmaj please keep in xxunk that the new book comes with to xxunk cd 's that are xxup"
6,i 'm not talking about the xxup cd ) . xxmaj diffe

In [27]:
dls.dataset.

<bound method Tokenizer.get_lengths of Tokenizer:
encodes: (Path,object) -> encodes
(str,object) -> encodes
decodes: (object,object) -> decodes
>

In [16]:
learn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [19]:
learn.load_encoder('/home/advaitmb/notebooks/projects/PreDiction/nbs/amazon_reviews_enc')

RuntimeError: Error(s) in loading state_dict for AWD_LSTM:
	size mismatch for encoder.weight: copying a param with shape torch.Size([28145, 400]) from checkpoint, the shape in current model is torch.Size([60008, 400]).
	size mismatch for encoder_dp.emb.weight: copying a param with shape torch.Size([28145, 400]) from checkpoint, the shape in current model is torch.Size([60008, 400]).
	size mismatch for rnns.0.weight_hh_l0_raw: copying a param with shape torch.Size([4600, 1150]) from checkpoint, the shape in current model is torch.Size([4608, 1152]).
	size mismatch for rnns.0.module.weight_ih_l0: copying a param with shape torch.Size([4600, 400]) from checkpoint, the shape in current model is torch.Size([4608, 400]).
	size mismatch for rnns.0.module.bias_ih_l0: copying a param with shape torch.Size([4600]) from checkpoint, the shape in current model is torch.Size([4608]).
	size mismatch for rnns.0.module.bias_hh_l0: copying a param with shape torch.Size([4600]) from checkpoint, the shape

In [15]:
learn.fit_one_cycle(1, 1e-2)

KeyboardInterrupt: 